In [1]:
import os

In [2]:
%pwd

'd:\\Study\\Python\\ML\\car_price_prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Study\\Python\\ML\\car_price_prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [17]:
from car_price_prediction.constants import *
from car_price_prediction.utils.common import read_yaml, create_directories

In [18]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [20]:
import os
import urllib.request as request
import zipfile
from car_price_prediction import logger

In [30]:
import os
import zipfile
from urllib.request import urlretrieve
from car_price_prediction import logger
from car_price_prediction.entity.config_entity import DataIngestionConfig
from pathlib import Path

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with the following info: \n{headers}")
        else:
            logger.info(f"File already exists with size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"Extracted zip file to {unzip_path}")

In [32]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-07-11 14:48:44,466: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-11 14:48:44,466: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-11 14:48:44,466: INFO: common: created directory at: artifacts]
[2025-07-11 14:48:44,466: INFO: common: created directory at: artifacts/data_ingestion]
[2025-07-11 14:48:46,913: INFO: 883321432: artifacts/data_ingestion/data.zip downloaded with the following info: 
Connection: close
Content-Length: 439363
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "f02129725677995f2b90e6feee5c59cb0cc92525384c3ad1f4f2bc5010823136"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F2D7:93E75:76352:16D2F3:6870D6EC
Accept-Ranges: bytes
Date: Fri, 11 Jul 2025 09:18:37 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4731-BOM
X-Cache: M